<p><strong><font size="6">WALOUS</font></strong></p>

<p><strong><font size="6">Fusion LC classifications results</font></strong></p>

<p><strong><font size="6">Import data and preprocessing</font></strong></p>

This python code implement the method developed by ANAGEO (ULB). 

Code developped on Linux Mint 18.1 (Ubuntu Xenial 16.04) and GRASS GIS 7.3.svn (r71315).

# Table of Contents

<div id="toc"></div>

The following cell is a Javascript section of code for building the Jupyter notebook's table of content.

In [1]:
%%javascript
$.getScript('https://kmahelona.github.io/ipython_notebook_goodies/ipython_notebook_toc.js')

<IPython.core.display.Javascript object>

# Define working environment

**Import libraries**

In [2]:
# Import libraries needed for setting parameters of operating system 
import os
import sys
import csv
import tempfile
import glob
import math
import pickle

In [3]:
## Import multiprocessing and functools libraries
import multiprocessing
from multiprocessing import Pool
from functools import partial

** Add folder with SCR provided belong to this notebook**

In [4]:
# Add local module to the path
src = os.path.abspath('../SRC')
if src not in sys.path:
    sys.path.append(src)

** Setup environment variables for TAIS DESKTOP (Linux Mint + GRASS Dev) **

Please edit the file in `../SRC/config.py`, containing the configuration parameters, according to your own computer setup. The following cell is used to run this file.



In [5]:
run ../SRC/config.py

In [6]:
print config_parameters

{'outputfolder_classifRF_tiles': '../../../Results/Classif_RF/classif_tiles', 'outputfolder_classifRF': '../../../Results/Classif_RF', 'outputfolder_Logfile': '../../../Results/Log_file', 'permanent_mapset': 'PERMANENT', 'outputfolder_training_sample': '../../../Results/Training_sample', 'locationepsg': '31370', 'outputfolder': '../../../Results', 'list_tiles': '../../../Results/list_tiles', 'pixel_classes_list': '../../../Results/pixel_classes_list', 'outputfolder_classifRF_csv': '../../../Results/Classif_RF/classif_csv', 'gisdb': '../../GRASSDATA', 'location': 'WALOUS_31370', 'outputfolder_classfeatures': '../../../Results/Classification_features', 'PYTHONLIB': '/usr/lib/python2.7', 'njobs': 6, 'rf_trained_model': '../../../Results/Classif_RF/rf_trained_model.rda', 'GISBASE': '/usr/lib/grass76'}


In [7]:
# Import functions that setup the environmental variables
import environ_variables as envi

In [8]:
# Set environmental variables
envi.setup_environmental_variables() 
# Display current environment variables of your computer
envi.print_environmental_variables()

MDMSESSION = mate 	
MANDATORY_PATH = /usr/share/gconf/mate.mandatory.path 	
MATE_DESKTOP_SESSION_ID = this-is-deprecated 	
LESSOPEN = | /usr/bin/lesspipe %s 	
MDM_LANG = fr_BE.UTF-8 	
LOGNAME = tais 	
USER = tais 	
HOME = /home/tais 	
XDG_VTNR = 8 	
PATH = /usr/local/bin:/home/tais/BIN:/home/tais/bin:/home/tais/.local/bin:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin:/usr/games:/usr/local/games:/usr/lib/grass76/bin:/usr/lib/grass76/script:/usr/lib/grass76/lib 	
CLICOLOR = 1 	
DISPLAY = :0.0 	
SSH_AGENT_PID = 1986 	
LANG = fr_BE.UTF-8 	
TERM = xterm-color 	
SHELL = /bin/bash 	
GIS_LOCK = $$ 	
XAUTHORITY = /home/tais/.Xauthority 	
SESSION_MANAGER = local/tais-HP-Z620-Workstation:@/tmp/.ICE-unix/1915,unix/tais-HP-Z620-Workstation:/tmp/.ICE-unix/1915 	
SHLVL = 1 	
QT_LINUX_ACCESSIBILITY_ALWAYS_ON = 1 	
INSIDE_CAJA_PYTHON =  	
QT_ACCESSIBILITY = 1 	
LD_LIBRARY_PATH = :/usr/lib/grass76/lib 	
COMPIZ_CONFIG_PROFILE = mate 	
WINDOWPATH = 8 	
GTK_OVERLAY_SCROLLING = 0 	
PYTHONPATH

** GRASS GIS Python libraries **

In [9]:
# Import libraries needed to launch GRASS GIS in the jupyter notebook
import grass.script.setup as gsetup
# Import libraries needed to call GRASS using Python
import grass.script as gscript

** Other functions**

In [10]:
# Import function that check existance and create GRASS GIS database folder if needed
from grass_database import check_gisdb, check_location, check_mapset, working_mapset
# Import functions for processing time information
from processing_time import start_processing, print_processing_time
# Import function that generate a random name in the GRASS GIS environement
from random_layer_name import random_layer_name
# Import function that check and create folder
from mkdir import check_create_dir
# Import function that check if GRASS GIS add-on is installed and install it if needed
from gextension import check_install_addon
# Import function that change .csv delimiter
from ChangeCsvDelimiter import ChangeCsvDelimiter
# Import function for .gzip archive management
from gzip_management import decompress_gzip
# Import function for getting list of class of a raster
from data_prep import data_prep
# Import function the allow sorting strings with number as if they was number (natural order)
from sorting_natural import natural_keys

**-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-**

In [11]:
def launch_mapset(mapset):
    #Declare empty list that will contain the messages to return
    return_message = []
    # Check if the location exists and create it if not, with the CRS defined by the epsg code 
    return_message.append(check_location(config_parameters["gisdb"],config_parameters['location'],config_parameters["locationepsg"]))
    # Check if mapset exists
    return_message.append(check_mapset(config_parameters["gisdb"],config_parameters['location'],mapset))
    # Change the current working GRASS GIS session mapset
    return_message.append(working_mapset(config_parameters["gisdb"],config_parameters['location'],mapset))
    # Return
    return return_message

**-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-**

## Create new directories

In [12]:
# Check and create folder if needed
check_create_dir(config_parameters['outputfolder_training_sample'])

The folder '../../../Results/Training_sample' has been created


# Create mapsets with input data for the processing

## OBIA land cover

In [17]:
# Create mapset 
launch_mapset("OBIA")

["Location 'WALOUS_31370' already exist",
 "'OBIA' mapset already exists in location 'WALOUS_31370'",
 "You are now working in mapset 'WALOUS_31370/OBIA'"]

In [14]:
# Create a list of paths to files with a specific name prefix
list_of_obia_raster = []
list_file = glob.glob(os.path.join(data['obia_folder'],"SEGMENTS","segs_tile_*.tif"))
print "There are %s paths in the list"%len(list_file)

There are 374 paths in the list


In [15]:
# Create a list of tuple with GRASS GIS layer name for segmentation raster and their path in the computer drive 
[list_of_obia_raster.append((os.path.splitext(a)[0].split(os.sep)[-1],a)) for a in list_file]
for a in list_of_obia_raster:
    print a

('segs_tile_2484', '../../../Data/obia_2018/SEGMENTS/segs_tile_2484.tif')
('segs_tile_3510', '../../../Data/obia_2018/SEGMENTS/segs_tile_3510.tif')
('segs_tile_5792', '../../../Data/obia_2018/SEGMENTS/segs_tile_5792.tif')
('segs_tile_5337', '../../../Data/obia_2018/SEGMENTS/segs_tile_5337.tif')
('segs_tile_2666', '../../../Data/obia_2018/SEGMENTS/segs_tile_2666.tif')
('segs_tile_3872', '../../../Data/obia_2018/SEGMENTS/segs_tile_3872.tif')
('segs_tile_6194', '../../../Data/obia_2018/SEGMENTS/segs_tile_6194.tif')
('segs_tile_1843', '../../../Data/obia_2018/SEGMENTS/segs_tile_1843.tif')
('segs_tile_1197', '../../../Data/obia_2018/SEGMENTS/segs_tile_1197.tif')
('segs_tile_1253', '../../../Data/obia_2018/SEGMENTS/segs_tile_1253.tif')
('segs_tile_5920', '../../../Data/obia_2018/SEGMENTS/segs_tile_5920.tif')
('segs_tile_5758', '../../../Data/obia_2018/SEGMENTS/segs_tile_5758.tif')
('segs_tile_4003', '../../../Data/obia_2018/SEGMENTS/segs_tile_4003.tif')
('segs_tile_5464', '../../../Data/obia

In [18]:
# Delete existing segmentation raster from previous versions processings
for rast in list_of_obia_raster:
    gscript.run_command('g.remove', flags='f', type='raster', name=rast[0])

In [19]:
# Import individual segmentation rasters (for each tile)
start_import = start_processing()
for rast in list_of_obia_raster:
    gscript.run_command('r.in.gdal', overwrite=True, input=rast[1] , output=rast[0])
print_processing_time(start_import, "Import achieved in ")

In [30]:
# Create virtual raster
gscript.run_command('r.buildvrt', overwrite=True, 
                    input=",".join([a[0] for a in list_of_obia_raster]), 
                    output='segmentation')

0

## Pixel ortho land cover

In [22]:
# Create mapset 
launch_mapset("PIXEL_ORTHO_LC")

["Location 'WALOUS_31370' already exist",
 "'PIXEL_ORTHO_LC' mapset created in location 'WALOUS_31370'",
 "You are now working in mapset 'WALOUS_31370/PIXEL_ORTHO_LC'"]

In [23]:
list_of_raster = []
list_file = glob.glob(os.path.join(data['ortho_lc'][1],"*_classif_pixels2018_erp_2.tif"))  # One raster per test zone
[list_of_raster.append((os.path.splitext(a)[0].split(os.sep)[-1],a)) for a in list_file]
list_of_raster

[('charleroi2_classif_pixels2018_erp_2',
  '../../../Data/pixel_2018/charleroi2_classif_pixels2018_erp_2.tif'),
 ('herve2_classif_pixels2018_erp_2',
  '../../../Data/pixel_2018/herve2_classif_pixels2018_erp_2.tif'),
 ('marche_classif_pixels2018_erp_2',
  '../../../Data/pixel_2018/marche_classif_pixels2018_erp_2.tif'),
 ('arlon_classif_pixels2018_erp_2',
  '../../../Data/pixel_2018/arlon_classif_pixels2018_erp_2.tif'),
 ('charleroi_classif_pixels2018_erp_2',
  '../../../Data/pixel_2018/charleroi_classif_pixels2018_erp_2.tif'),
 ('herve_classif_pixels2018_erp_2',
  '../../../Data/pixel_2018/herve_classif_pixels2018_erp_2.tif')]

In [24]:
# Declare list that will contain tiles
list_of_tile_raster = []

# Create virtual raster
for input_data in list_of_raster:
    # Define tile number (average)
    tile_number = 200
    # Import raster    #Only first band that contain the first clas
    gscript.run_command('r.in.gdal', overwrite=True, input=input_data[1] , output=input_data[0], band=1)  
    # Get raster information
    info = gscript.raster_info(input_data[0])
    # Tile raster 
    gscript.run_command('r.tile', overwrite=True, input=input_data[0], output="%s_tile"%input_data[0],
                        width=math.ceil(int(info.cols)/math.sqrt(tile_number)), 
                        height=math.ceil(int(info.rows)/math.sqrt(tile_number)))
    # Get a list of tiles name
    tiled_list = gscript.list_strings(type='raster',pattern="%s_tile*"%input_data[0])
    [list_of_tile_raster.append(tile) for tile in tiled_list]
    # Remove original raster
    gscript.run_command('g.remove', flags='f',  type="raster", name=input_data[0])

# Create virtual raster
gscript.run_command('r.buildvrt', overwrite=True, 
                    input=",".join(list_of_tile_raster), 
                    output=data['ortho_lc'][0])

0

In [25]:
# Get list of categories 
start_parallel = start_processing()
# Parallel jobs
p = Pool(20)
output = p.map(data_prep, list_of_tile_raster)  # Launch the processes for as many items in the list (if function with a return, the returned results are ordered thanks to 'map' function)
p.close()
p.join()
# Get unique categories
output_2 = [cl for sublist in output for cl in sublist[2]]
class_dict = {a:'' for a in output_2}
pixel_classes_list = class_dict.keys()
pixel_classes_list.sort(key=float)
# Export python object to file
with open(config_parameters['pixel_classes_list'], 'wb') as outputfile:
  pickle.dump(pixel_classes_list, outputfile)
# Print
print_processing_time(start_parallel, "Computation achieved in ")

'Computation achieved in 10.1 seconds'

In [26]:
pixel_classes_list

['2', '5', '6', '11', '12', '31', '32', '41', '42', '91', '92', '93', '94']

## Pixel SENTINEL (land cover and Sent2Agri)

In [27]:
# Create mapset 
launch_mapset("PIXEL_SENTINEL")

["Location 'WALOUS_31370' already exist",
 "'PIXEL_SENTINEL' mapset created in location 'WALOUS_31370'",
 "You are now working in mapset 'WALOUS_31370/PIXEL_SENTINEL'"]

In [28]:
# Define input data
inputs = [data['senti_lc'], data['senti_croptype']]
#inputs = [data['senti_lc'],]

# Define tile number (average)
tile_number = 1500

# Create VRT
for input_data in inputs:
    # Import raster 
    gscript.run_command('r.in.gdal', overwrite=True, input=input_data[1] , output=input_data[0])
    # Get raster information
    info = gscript.raster_info(input_data[0])
    # Tile raster 
    gscript.run_command('r.tile', overwrite=True, input=input_data[0], output="%s_tile"%input_data[0],
                        width=math.ceil(int(info.cols)/math.sqrt(tile_number)), 
                        height=math.ceil(int(info.rows)/math.sqrt(tile_number)))
    # Get a list of tiles name
    list_of_tile_raster = gscript.list_strings(type='raster',pattern="%s_tile*"%input_data[0])
    # Create virtual raster
    gscript.run_command('r.buildvrt', overwrite=True, 
                        input=",".join(list_of_tile_raster),
                        output=input_data[0])

## Binaries from vectorial reference data (Squelette vectoriel, SIGEC, masque forestier)

In [ ]:
# Create mapset 
launch_mapset("BINARIES")

**SIGEC agricultural plots + GEMBLOUX forest mask**

In [ ]:
# Define input data
inputs = [data['binary_agri'], data['binary_forest']]
# Define tile number (average)
tile_number = 600

# Create VRT
for input_data in inputs:
    # Import raster 
    gscript.run_command('r.in.gdal', overwrite=True, input=input_data[1] , output=input_data[0])
    # Get raster information
    info = gscript.raster_info(input_data[0])
    # Tile raster 
    gscript.run_command('r.tile', overwrite=True, input=input_data[0], output="%s_tile"%input_data[0],
                        width=math.ceil(int(info.cols)/math.sqrt(tile_number)), 
                        height=math.ceil(int(info.rows)/math.sqrt(tile_number)))
    # Get a list of tiles name
    list_of_tile_raster = gscript.list_strings(type='raster',pattern="%s_tile*"%input_data[0])
    # Create virtual raster
    gscript.run_command('r.buildvrt', overwrite=True, 
                        input=",".join(list_of_tile_raster),
                        output=input_data[0])

**Vectorial skeletton (road network, hydrological ways, buildings)**

In [ ]:
# Define input data
inputs = [data['binary_roadnet'], data['binary_hydro'], data['binary_built']]
# Define tile number (average)
tile_number = 1500

# Create VRT
for input_data in inputs:
    # Import raster 
    gscript.run_command('r.in.gdal', overwrite=True, input=input_data[1] , output=input_data[0])
    # Get raster information
    info = gscript.raster_info(input_data[0])
    # Tile raster 
    gscript.run_command('r.tile', overwrite=True, input=input_data[0], output="%s_tile"%input_data[0],
                        width=math.ceil(int(info.cols)/math.sqrt(tile_number)), 
                        height=math.ceil(int(info.rows)/math.sqrt(tile_number)))
    # Get a list of tiles name
    list_of_tile_raster = gscript.list_strings(type='raster',pattern="%s_tile*"%input_data[0])
    # Create virtual raster
    gscript.run_command('r.buildvrt', overwrite=True, 
                        input=",".join(list_of_tile_raster),
                        output=input_data[0])

## Other data (Tiles vector layer, training points)

In [29]:
# Create mapset 
launch_mapset("PERMANENT")

["Location 'WALOUS_31370' already exist",
 "'PERMANENT' mapset already exists in location 'WALOUS_31370'",
 "You are now working in mapset 'WALOUS_31370/PERMANENT'"]

In [30]:
# Allow access to other mapsets
gscript.run_command('g.mapsets', mapset='OBIA', operation='add')

0

**Import tiles of OBIA processing**

In [31]:
# Import vector
gscript.run_command('v.in.ogr', overwrite=True, 
                    input=data['tiles'][1], output=data['tiles'][0], 
                    layer=data['tiles_gpkg_layer'], key='cat')

0

In [32]:
# Return cat values of tile layer
a = gscript.parse_command('v.db.select', flags='c', map=data['tiles'][0], columns='cat').keys()
a.sort(key=natural_keys)
tile_list = a

In [33]:
# Export python object to file
with open(config_parameters['list_tiles'], 'wb') as outputfile:
  pickle.dump(tile_list, outputfile)

**Import training points, get value of segment ID and tile ID and export as .csv file**

In [ ]:
# Import vector
gscript.run_command('v.in.ogr', overwrite=True, input=data['training_points'][1], output=data['training_points'][0])
gscript.run_command('g.region', raster='segmentation')
# Get only point within tile vector 
gscript.run_command('v.clip', overwrite=True, input='training', clip='tiles', output='training_tmp')
gscript.run_command('g.rename', overwrite=True, vector='training_tmp,training')

In [ ]:
# Add seg_id in attribute table
gscript.run_command('v.db.addcolumn', map=data['training_points'][0], columns="seg_id integer")
gscript.run_command('v.what.rast', map=data['training_points'][0], raster='segmentation', column='seg_id')
# Add Tile ID in attribute table
gscript.run_command('v.db.addcolumn', map=data['training_points'][0], columns="tile_id integer")

In [ ]:
gscript.run_command('v.what.vect', map=data['training_points'][0], column='tile_id', 
                    query_map=data['tiles'][0], query_column='cat')
# Export attribute table as .csv
output_file = os.path.join(config_parameters['outputfolder_training_sample'],'training_points_with_segid.csv')
gscript.run_command('v.db.select', overwrite=True, 
                    map=data['training_points'][0], separator='comma', file=output_file)